 #  A Whale off the Port(folio)

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:
1. whale_returns.csv
2. algo_returns.csv
3. sp500_history.csv

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from pathlib import Path

%matplotlib inline

# Funtion purpose is create csv dataframes in a consistant format.
# Function returns a data frame object based on the supplied file name and column to be indexed on.
# To retain consistant column formats, the column index is tested for a "Date" value
# If supplied value is true, the column index is set to the Date column
# If supplied value is false, assumed the column index is set to Trade DATE
# also Close column is renamed to "NOCP"
# Dataframe is sorted based on index in ascending order

def prep_dataframe_csv(file,col_index):
    
    filepath = Path("Resources/" + file) 
    
    if col_index == "Date":
        df_csv = pd.read_csv(filepath, parse_dates=True, index_col="Date", infer_datetime_format=True)
        # Sort DataFrame
        df_csv.sort_values("Date", ascending=True, inplace=True)
    else:
        df_csv = pd.read_csv(filepath, parse_dates=True, index_col="Trade DATE", infer_datetime_format=True)
        
        # Sort DataFrame and rename Close column
        df_csv = df_csv.rename(columns ={"Close":"NOCP"})
        df_csv.sort_values("Trade DATE", ascending=True, inplace=True)
    return df_csv

# Function purpose it to apply column modification to keep portfolio dataframes consistant.
# Function returns a data frame object based with the following modification.
# Symbol column is added next to index column.
# Close column is renamed to NOCP.
# Index name is relabled as Trade DATE.

def prep_dataframe_columns(df,stock):
    
    df.insert(0,"Symbol", stock)
    df = df.rename(columns ={"Close":"NOCP"})
    df = df.rename_axis("Trade DATE")

    return df

In [ ]:
# Reading whale returns
# Call function to generate DataFrame
whale_returns = prep_dataframe_csv("whale_returns.csv","Date")

whale_returns.head()

In [ ]:
# Count nulls
whale_returns.isnull().sum()
#whale_returns.duplicated()

In [ ]:
# Drop nulls
whale_returns.dropna(inplace=True)
whale_returns.isnull().sum()

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
# Call function to generate DataFrame
algo_returns = prep_dataframe_csv("algo_returns.csv","Date")

algo_returns.head()

In [ ]:
# Count nulls
algo_returns.isnull().sum()
#algo_returns.duplicated()

In [ ]:
# Drop nulls
algo_returns.dropna(inplace=True)
algo_returns.isnull().sum()

## S&P 500 Returns

Read the S&P500 Historic Closing Prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
# Call function to generate DataFrame
sp500_price_hist = prep_dataframe_csv("sp500_history.csv","Date")

sp500_price_hist.head()

In [ ]:
# Check Data Types
sp500_price_hist.dtypes

In [ ]:
# Fix Data Types
sp500_price_hist["Close"] = sp500_price_hist["Close"].str.replace("$","")
sp500_price_hist["Close"] = sp500_price_hist["Close"].astype("float")
sp500_price_hist.dtypes
#sp500_price_hist.head()

In [ ]:
# Calculate Daily Returns
sp500_returns = sp500_price_hist.pct_change()
sp500_returns.head()

In [ ]:
# Drop nulls
sp500_returns.dropna(inplace=True)
sp500_returns.head()

In [ ]:
# Rename Column
sp500_returns.rename(columns ={"Close":"S&P 500"},inplace=True)
sp500_returns.head()

## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Concatenate all DataFrames into a single DataFrame
portfolio_returns = pd.concat([whale_returns,algo_returns,sp500_returns], axis="columns", join="inner")
portfolio_returns.head()

# Portfolio Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance

Calculate and Plot the daily returns and cumulative returns. Does any portfolio out perform the S&P 500?


It is difficult to determine which portfolio is out performing the S&P 500 from the Daily Returns plot.  At the onset they all appear to follow the S&P 500 with TIGER GLOBAL MANAGEMENT LLC showing large spikes in early 2017 and 2019.  More analysis is required.  When looking at the Cummulative Returns plot, Aglo 2 follows the S&P 500 the closest where Algo 1 and Berkshire Hathaway INC are out perfroming the S&P 500  

In [ ]:
# Plot daily returns
portfolio_returns.plot(figsize=(20,10), title='Daily Returns Comparsion')

In [ ]:
# Plot cumulative returns (cumu)
portfolio_returns_cumu = (1 + portfolio_returns).cumprod() 
portfolio_returns_cumu.plot(figsize=(20,10), title='Cumulative Returns Comparison')

## Risk

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

While the median values in the box plots for all the porfilios are roughly the same centered around zero, each portfolio appears to be normally distributed with their median values falling in the middle of each box. Portfolio with the most dispersed data (largest box) is Berkshire Hathaway INC and portfolios with the least dispersed data (smallest box) are Algo 1, S&P 500 and Paulson & CO all very close.  When looking and the data spread, Berkshire Hathaway INC has the largest difference between its min and max values and Algo 1 with smallest.  Portfolio's that are riskier, having standard deviations greater than than the S&P 500 are:

BERKSHIRE HATHAWAY INC      = 0.012919

TIGER GLOBAL MANAGEMENT LLC = 0.010894

Lowest risk is:
PAULSON & CO.INC.  = 0.007023079026076193

In [ ]:
# Box plot to visually show risk. Which box has the largest spread? Which has the smallest spread?
portfolio_boxplot = portfolio_returns.plot(kind='box',figsize=(20,10), title='Portfolio Risk Comparison')
portfolio_boxplot.set_xticklabels(portfolio_boxplot.get_xticklabels(), rotation=45, horizontalalignment='right')
portfolio_boxplot

In [ ]:
# Daily Standard Deviations
# Calculate the standard deviation for each portfolio. 
portfolio_returns_std = portfolio_returns.std()

portfolio_returns_std

In [ ]:
# Which portfolios are riskier than the S&P 500?
portfolio_returns_std.loc[portfolio_returns_std >= portfolio_returns_std['S&P 500']].sort_values(ascending=False)

In [ ]:
#Highest risk
max_risk_portfolio = portfolio_returns_std[portfolio_returns_std == portfolio_returns_std.max()].index[0]
max_risk_value = portfolio_returns_std.max()
print(f" Portfolio with the highest risk:{max_risk_portfolio} = {max_risk_value}")

In [ ]:
#Lowest risk
min_risk_portfolio = portfolio_returns_std[portfolio_returns_std == portfolio_returns_std.min()].index[0]
min_risk_value = portfolio_returns_std.min()
print(f" Portfolio with the lowesr risk: {min_risk_portfolio} = {min_risk_value}")

In [ ]:
# Determine which portfolios are riskier than the S&P 500  - My solution
#portfolio_returns_rsky = pd.Series(portfolio_returns_std.values > portfolio_returns_std['S&P 500'])
#portfolio_returns_rsky.index=portfolio_returns_std.index
#portfolio_returns_rsky

In [ ]:
# Determine which portfolios are riskier than the S&P 500  - TA assisted solution
portfolio_returns_std > portfolio_returns_std['S&P 500']


In [ ]:
# Calculate the annualized standard deviation (252 trading days)
portfolio_returns_std_ann = portfolio_returns_std * np.sqrt(252)
portfolio_returns_std_ann

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Plot the rolling standard deviation of the various portfolios along with the rolling standard deviation of the S&P 500 (consider a 21 day window). Does the risk increase for each of the portfolios at the same time risk increases in the S&P?
2. Construct a correlation table for the algorithmic, whale, and S&P 500 returns. Which returns most closely mimic the S&P?
3. Choose one portfolio and plot a rolling beta between that portfolio's returns and S&P 500 returns. Does the portfolio seem sensitive to movements in the S&P 500?
4. An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the ewm with a 21 day half-life.

Upon observing the 21 day rolling standard deviation plot, risk increases for Soros Fund, Algo 1 and Algo 2 follow the S&P 500. Tiger Global follows closely as well with the exception of some sharp spikes in early 2017 and 2018.  Berkshire follows but at a high value, however Paulson & Co follows at a lower value.

The returns that closely mimic the S&P 500 is the Algo 2 with a positive correlation of 0.858764.  A close second is Soro's Fund with a positive correlation of 0.837864

As a comparison, Berkshire Hathaway was chosen to combare beta values against the S&P 500.  Berkshire does appear sensitive to the S&P with a Beta value of 1.1347891297491723 which means it follows the S&P 500 13% higher value.

In [ ]:
# Calculate and plot the rolling standard deviation for
# the S&P 500 and whale portfolios using a 21 trading day window
portfolio_returns.sort_index(inplace=True)
portfolio_returns.rolling(window=21).std().plot(figsize=(20,10), title='21 Day Rolling Standard Deviation Comparison')

In [ ]:
# Construct a correlation table
portfolio_corr = portfolio_returns.corr()
portfolio_corr

In [ ]:
# Heatmap of correlations  Negative Correlation (Red) Positive (Blue)
sns.heatmap(portfolio_corr, vmin=-1, vmax=1,cmap=sns.diverging_palette(20, 220, n=200),annot = True)

In [ ]:
# Calculate Beta for a single portfolio compared to the total market (S&P 500)
# (Your graph may differ, dependent upon which portfolio you are comparing)
variance = portfolio_returns['S&P 500'].var()
covariance = portfolio_returns['BERKSHIRE HATHAWAY INC'].cov(portfolio_returns['S&P 500'])
berk_beta = covariance/variance
berk_beta

In [ ]:
#Calculate 21-Day Rolling covariance of BERKSHIRE HATHAWAY INC Returns vs. S&P 500 Returns
berk_roll_cov = portfolio_returns['BERKSHIRE HATHAWAY INC'].rolling(window=21).cov(portfolio_returns['S&P 500'])

# Calculate 21-day rolling variance of S&P 500 Returns 
berk_roll_var = portfolio_returns['S&P 500'].rolling(window=21).var()

#Calculate rolling beta for BERKSHIRE HATHAWAY INC Returns
berk_roll_beta = berk_roll_cov / berk_roll_var

#Plot of 21-day BETA of BERKSHIRE HATHAWAY INC
berk_roll_beta.plot(figsize=(20, 10), title='Rolling 21-Day BETA for BERKSHIRE HATHAWAY INC Returns vs. S&P 500 Returns')

In [ ]:
#Plot of 21-day variance of BERKSHIRE HATHAWAY INC
berk_roll_var.plot(figsize=(20, 10), title='Rolling 21-Day Variance for BERKSHIRE HATHAWAY INC Returns vs. S&P 500 Returns')

In [ ]:
#Plot of 21-day covariance of BERKSHIRE HATHAWAY INC
berk_roll_cov.plot(figsize=(20, 10), title='Rolling 21-Day Covariance for BERKSHIRE HATHAWAY INC Returns vs. S&P 500 Returns')

In [ ]:
# Calculate a rolling window using the exponentially weighted moving average. 
portfolio_returns_ewm = portfolio_returns.ewm(halflife=21).std()
#portfolio_returns_ewm.head()
portfolio_returns_ewm.plot(figsize=(20, 10))

---

## Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. (After all, if you could invest in one of two portfolios, each offered the same 10% return, yet one offered lower risk, you'd take that one, right?)

1. Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot.
2. Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

***On the basis of this performance metric, do our algo strategies outperform both 'the market' and the whales? Type your answer here:***

From the Sharpe Ratios and bar plot, Algo 1 out performs both the S&P 500 and all the Whale portfolios.  Berkshire Hathaway, Whale portfolio's top performer only slightly out performed Algo 2.

In [ ]:
# Calculate annualized Sharpe Ratios (shrpr)

portfolio_returns_shrpr = (portfolio_returns.mean()*252)/(portfolio_returns_std*np.sqrt(252))
portfolio_returns_shrpr

In [ ]:
# Visualize the sharpe ratios as a bar plot
port_boxplot = portfolio_returns_shrpr.plot(kind='bar',figsize=(10,5), title="Sharpe Ratios Comparison")
port_boxplot.set_xticklabels(port_boxplot.get_xticklabels(), rotation=45, horizontalalignment='right')
port_boxplot

---

# Portfolio Returns

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Visit [Google Sheets](https://docs.google.com/spreadsheets/) and use the in-built Google Finance function to choose 3-5 stocks for your own portfolio.
2. Download the data as CSV files and calculate the portfolio returns.
3. Calculate the returns for each stock.
4. Using those returns, calculate the weighted returns for your entire portfolio assuming an equal number of shares for each stock.
5. Add your portfolio returns to the DataFrame with the other portfolios and rerun the analysis. How does your portfolio fair?


## Your analysis should include the following:

- Using all portfolios:
 - The annualized standard deviation (252 trading days) for all portfolios.
 - The plotted rolling standard deviation using a 21 trading day window for all portfolios.
 - The calculated annualized Sharpe Ratios and the accompanying bar plot visualization.
 - A correlation table.
- Using your custom portfolio and one other of your choosing:
 - The plotted beta. 

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Read the first stock - Google
# Call function to generate DataFrame
GOOG_price_hist = prep_dataframe_csv("goog_historical.csv", "Trade DATE")

# Sort DataFrame
GOOG_price_hist.sort_values("Trade DATE", ascending=True, inplace=True)
GOOG_price_hist.head()

In [ ]:
# Read the second stock - Apple
# Call function to generate DataFrame
AAPL_price_hist = prep_dataframe_csv("aapl_historical.csv","Trade DATE")

AAPL_price_hist.head()

In [ ]:
# Read the third stock - Costco
# Call function to generate DataFrame
COST_price_hist = prep_dataframe_csv("cost_historical.csv","Trade DATE")

COST_price_hist.head()

In [ ]:
# Read the forth stock - Shopify - Data supplied by Google Finance Function and Google Sheets
# Call function to generate DataFrame
SHOP_price_hist = prep_dataframe_csv("shop_historical.csv","Date")

# Format DataFrame to be consistant with other DataFrame, add Symbol column
SHOP_price_hist = prep_dataframe_columns(SHOP_price_hist,"SHOP")

SHOP_price_hist.head()

In [ ]:
# Read the fifth stock - Intuit - - Data supplied by Google Finance Function and Google Sheets
# Call function to generate DataFrame
INTU_price_hist = prep_dataframe_csv("intu_returns.csv","Date")

# Format DataFrame to be consistant with other DataFrame, add Symbol column
INTU_price_hist = prep_dataframe_columns(INTU_price_hist,"INTU")

INTU_price_hist.head()

In [ ]:
# Concatenate all stocks into a single DataFrame
my_portfolio_hist = pd.concat([GOOG_price_hist, AAPL_price_hist, COST_price_hist, SHOP_price_hist,INTU_price_hist], axis="rows", join="inner")
my_portfolio_hist.head()

In [ ]:
# Reset the index
my_portfolio_hist = my_portfolio_hist.reset_index()
my_portfolio_hist.head()

In [ ]:
# Pivot so that each column of prices represents a unique symbol
my_portfolio_hist = my_portfolio_hist.pivot(columns="Symbol", values="NOCP", index="Trade DATE").copy()
my_portfolio_hist.head()

In [ ]:
# Drop Nulls
my_portfolio_hist.dropna(inplace=True)
my_portfolio_hist.head()

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Calculate Daily Returns
my_portfolio_returns = my_portfolio_hist.pct_change().copy()
my_portfolio_returns

# Calculate weighted portfolio returns
AAPL_weight = 0.2
COST_weight = 0.2
GOOG_weight = 0.2
INTU_weight = 0.2
SHOP_weight = 0.2

my_portfolio_returns = AAPL_weight * my_portfolio_returns["AAPL"] + COST_weight * my_portfolio_returns["COST"] + GOOG_weight * my_portfolio_returns["GOOG"] + INTU_weight * my_portfolio_returns["INTU"] + SHOP_weight * my_portfolio_returns["SHOP"]
my_portfolio_returns.rename_axis("Date", inplace=True)
my_portfolio_returns.head()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Add your "Custom" portfolio to the larger dataframe of fund returns
cust_portfolio_returns = pd.concat([portfolio_returns,my_portfolio_returns],axis="columns", join="inner")
cust_portfolio_returns.rename(columns={0:"Custom"},inplace=True)
cust_portfolio_returns.head()

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
cust_portfolio_returns.dropna(inplace=True)
cust_portfolio_returns.head()

## Re-run the performance and risk analysis with your portfolio to see how it compares to the others

## Performance

Again it is difficult to determine if the Custom portfolio is out perfoming the S&P 500 and the Whale Portfolios from the Daily Returns plot.  At the onset they all appear to follow the S&P 500 with TIGER GLOBAL MANAGEMENT LLC showing a large downward spike in early 2019.  More analysis is required.  When looking at the Cummulative Returns plot for the 2 year period, Paulson Co follows the S&P 500 the closest where Algo 2 and the Custom portfolio are out perfroming the S&P 500  

In [ ]:
# Plot daily returns
cust_portfolio_returns.plot(figsize=(20,10), title='Daily Returns Comparison of Custom Portfolio')

In [ ]:
# Plot cumulative returns (cumu)
cust_port_returns_cumu = (1 + cust_portfolio_returns).cumprod()-1
cust_port_returns_cumu.plot(figsize=(20,10), title='Cumulative Returns Comparison of Custom Portfolio')

## Risk

***It should be noted that from the previous analysis, the data sample spanned from 2015 to 2019.  Time frame of the custom portfolio analisys spans Jun 2018 to May 2019.  This will impact the results of the analysis.***

Median values in the box plots for our Custom portolio is higher then the S&P 500 but appears to be normally distributed with the median value falling in the middle of the box. Data has about the same dispersment as than that of Berkshire Hathaway but more dispersed the S&P 500. When looking at the data spread, Berkshire Hathaway INC has the same difference between its min and max values as our Custom portfolio. The Custom Portfolio now has the higher standard deviation that is greater than the S&P 500 :

Custom = 0.016014

BERKSHIRE HATHAWAY INC = 0.015569

TIGER GLOBAL MANAGEMENT LLC = 0.014648

Lowest risk is: PAULSON & CO.INC. = 0.007353

In [ ]:
# Risk
# Box plot to visually show risk
cust_port_boxplot = cust_portfolio_returns.plot(kind='box', figsize=(20,10), title='Custom Portfolio Risk Comparison')
cust_port_boxplot.set_xticklabels(cust_port_boxplot.get_xticklabels(), rotation=45, horizontalalignment='right')
cust_port_boxplot

In [ ]:
# Calculate the standard deviation for each portfolio. 
cust_port_returns_std = cust_portfolio_returns.std()
cust_port_returns_std

In [ ]:
# Which portfolios are riskier than the S&P 500?
cust_port_returns_std.loc[cust_port_returns_std >= cust_port_returns_std['S&P 500']].sort_values(ascending=False)

In [ ]:
#Highest risk
cust_max_risk_port = cust_port_returns_std[cust_port_returns_std == cust_port_returns_std.max()].index[0]
cust_max_risk_value = cust_port_returns_std.max()
print(f" Portfolio with the highest risk: {cust_max_risk_port} = {cust_max_risk_value}")

In [ ]:
#Lowest risk
cust_min_risk_port = cust_port_returns_std[cust_port_returns_std == cust_port_returns_std.min()].index[0]
cust_min_risk_value = cust_port_returns_std.min()
print(f" Portfolio with the lowesr risk:{cust_min_risk_port} = {cust_min_risk_value}")

## Rolling Statistics

Upon observing the 21 day rolling standard deviation plot, risk increases for Soros Fund, Algo 1 and Algo 2 follow the S&P 500. Tiger Global follows closely as well with the 
exception of a large sharp spike in 2019-03/04. Our Custom portfolio follows at a higher rate close with Berkshire Hathaway.

The returns that closely mimic the S&P 500 are the Soro's Fund with a positive correlation of 0.876981. A close second is Algo 1 Fund with a positive correlation of 0.875721.  Our Custom portfolio is positivily corralated with the S&P 500 with a value of 0.866738

For a comparison, Berkshire Hathaway, Algo 2 and the Custom portfolio was chosen to combare beta values against the S&P 500. Berkshire and the Custom portfolios are sensitive to the S&P with the Custom portfolio
Beta value of 1.4491106488679828 which means it follows the S&P 500 at a 44% higher value.  Beta values for Algo 2 is 0.8037439333709113 and BERKSHIRE HATHAWAY INC equals 1.3853682313573936. 

If we look at the 21-day Beta graph, we can see how similar our Custom portfolio and Berkshire Hathaway are, being more volitile than Algo 2 when compared with the S&P 500


In [ ]:
# Calculate and plot the rolling standard deviation for
# the S&P 500 and whale portfolios using a 21 trading day window
cust_portfolio_returns.sort_index(inplace=True)
cust_portfolio_returns.rolling(window=21).std().plot(figsize=(20,10), title='21 Day Rolling Standard Deviation Comparison of Custom Portfolio')

In [ ]:
# Construct a correlation table
cust_port_corr = cust_portfolio_returns.corr()
cust_port_corr

In [ ]:
# Heatmap of correlations  Negative Correlation (Red) Positive (Blue)
sns.heatmap(cust_port_corr, vmin=-1, vmax=1,cmap=sns.diverging_palette(20, 220, n=200),annot = True)

In [ ]:
# Calculate Beta for BERKSHIRE HATHAWAY INC, Custom and Algo 2 compared to the total market (S&P 500)

# Calculate variance of all daily returns of portfolios vs. S&P 500
cust_sp500_var = cust_portfolio_returns['S&P 500'].var()

In [ ]:
# Calculate covariance of all daily returns for BERKSHIRE HATHAWAY INC, Custom and Algo 2 vs. S&P 500
cust_covariance = cust_portfolio_returns['Custom'].cov(cust_portfolio_returns['S&P 500'])
algo_covariance = cust_portfolio_returns['Algo 2'].cov(cust_portfolio_returns['S&P 500'])
berk_covariance = cust_portfolio_returns['BERKSHIRE HATHAWAY INC'].cov(cust_portfolio_returns['S&P 500'])

In [ ]:
# Calculate beta of BERKSHIRE HATHAWAY INC, Custom and Algo 2 vs. S&P 500
cust_beta = cust_covariance / cust_sp500_var
algo_beta = algo_covariance / cust_sp500_var
berk_beta = berk_covariance / cust_sp500_var

print("Beta Values")
print(f"Custom: {cust_beta} | Algo 2: {algo_beta} | BERKSHIRE HATHAWAY INC: {berk_beta}")

In [ ]:
# Calculate 21-day rolling covariance of BERKSHIRE HATHAWAY INC, Custom and Algo 2 vs. S&P 500
cust_roll_cov = cust_portfolio_returns['Custom'].rolling(window=21).cov(cust_portfolio_returns['S&P 500'])
cust_algo_roll_cov = cust_portfolio_returns['Algo 2'].rolling(window=21).cov(cust_portfolio_returns['S&P 500'])
cust_berk_roll_cov = cust_portfolio_returns['BERKSHIRE HATHAWAY INC'].rolling(window=21).cov(cust_portfolio_returns['S&P 500'])

In [ ]:
#Plot of 21-day covariance of BERKSHIRE HATHAWAY INC
ax = cust_roll_cov.plot(figsize=(20, 10), title='Rolling 21-Day Covariance Comparison of Custom Portfolio')
cust_algo_roll_cov.plot(ax=ax)
cust_berk_roll_cov.plot(ax=ax)

# Set the legend of the figure
ax.legend(["Custom", "Algo 2", "BERKSHIRE HATHAWAY INC"])

In [ ]:
# Calculate 21-day rolling variance of S&P 500
cust_sp500_roll_var = cust_portfolio_returns['S&P 500'].rolling(window=21).var()

In [ ]:
# Calculate 21-day rolling beta of BERKSHIRE HATHAWAY INC, Custom and Algo 2 vs. S&P 500
cust_roll_beta = cust_roll_cov / cust_sp500_roll_var
cust_algo_roll_beta = cust_algo_roll_cov / cust_sp500_roll_var
cust_berk_roll_beta = cust_berk_roll_cov / cust_sp500_roll_var

In [ ]:
# Plot the different datasets on same graph.
ax = cust_roll_beta.plot(figsize=(20, 10), title='Rolling 21-Day Beta Comparios of Custom Portfolio')
cust_algo_roll_beta.plot(ax=ax)
cust_berk_roll_beta.plot(ax=ax)

# Set the legend of the figure
ax.legend(["Custom", "Algo 2", "BERKSHIRE HATHAWAY INC"])

## Sharpe Ratio

From the Sharpe Ratios and bar plot, Algo 1 still out performs both the S&P 500 and all the Whale portfolios including the Custom portfolio.  The Custom portfolio placed second.

In [ ]:
# Calculate annualized Sharpe Ratios (shrpr)

cust_port_returns_shrpr = (cust_portfolio_returns.mean()*252)/(cust_port_returns_std*np.sqrt(252))
cust_port_returns_shrpr.sort_values(ascending=False)

In [ ]:
# Visualize the sharpe ratios as a bar plot
cust_port_boxplot =cust_port_returns_shrpr.plot(kind='bar',figsize=(10,5), title="Sharpe Ratios Comparison of Custom Portfolio")
cust_port_boxplot.set_xticklabels(cust_port_boxplot.get_xticklabels(), rotation=45, horizontalalignment='right')
cust_port_boxplot